# LangChain - Chains & Memory

Credit: Coursera

Chains:
- SimpleSequentialChain
- SequentialChain
- RouterChain

In [169]:
import os
import json
import pandas as pd
from langchain import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.prompts import ChatPromptTemplate
from langchain.memory import SimpleMemory

In [4]:
with open('../configs/config.json', 'r') as f:
    config = json.load(f)

In [5]:
os.environ['OPENAI_API_TYPE']=config['OPENAI_API_TYPE']
os.environ['OPENAI_API_VERSION']=config['OPENAI_API_VERSION']
os.environ['OPENAI_API_BASE']=config['OPENAI_API_BASE']
os.environ['OPENAI_API_KEY']=config['OPENAI_API_KEY']

In [ ]:
# Define the LLM model
llm = AzureChatOpenAI(deployment_name=config['OPENAI_MODEL_NAME'], 
                       model_name="gpt-35-turbo", 
                       temperature=0.0)

## Chains

![Simple Seuqntial Chain](../images/sequential_chains.png)

In [74]:
file = '../data/Data.csv'
df = pd.read_csv(file, sep=',')

### LLMChain
Basic chain which combines a model and a prompt

In [75]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [76]:
llm = AzureChatOpenAI(deployment_name=config['OPENAI_MODEL_NAME'], 
                       model_name="gpt-35-turbo", 
                       temperature=0.9)

In [77]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [78]:
chain = LLMChain(llm=llm, prompt=prompt)

In [79]:
product = "Queen Size Sheet Set"
chain.run(product)

'Royal Bedding Company'

## SimpleSequentialChain

![Simple Seuqntial Chain](../images/simple_sequential_chain.png)

In [80]:
from langchain.chains import SimpleSequentialChain

In [160]:
# Template for Entity Extraction
entity_template = """{instruction}
The new iPhone model is set to be released next month.
It has been highly anticipated by Apple fans and is expected
to feature a larger screen and improved camera."""

entity_prompt_template = ChatPromptTemplate.from_template(entity_template)
entity_chain = LLMChain(llm=llm, prompt=entity_prompt_template)

In [161]:
# Template for text generation
text_template = """Please summarize the information about the product based on the following: {entities}"""

text_prompt_template = ChatPromptTemplate.from_template(text_template)
text_chain = LLMChain(llm=llm, prompt=text_prompt_template)

In [162]:
# Template for Sentiment Analysis
sentiment_template = """Please provide a sentiment for the following text and produce output as json: {text}"""

sentiment_prompt_template = ChatPromptTemplate.from_template(sentiment_template)
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt_template)

In [163]:
# Create Final chain
overal_simple_chain = SimpleSequentialChain(chains=[entity_chain, text_chain, sentiment_chain], 
                                            verbose=True)

In [164]:
overal_simple_chain.run("Extract the entities from the following text and produce output in the json format:")



> Entering new SimpleSequentialChain chain...
{
  "entities": [
    {
      "entity": "iPhone",
      "type": "product"
    },
    {
      "entity": "Apple",
      "type": "company"
    },
    {
      "entity": "next month",
      "type": "time"
    },
    {
      "entity": "larger screen",
      "type": "feature"
    },
    {
      "entity": "improved camera",
      "type": "feature"
    }
  ]
}
Apple is releasing a new iPhone with a larger screen and improved camera next month.
{
    "text": "Apple is releasing a new iPhone with a larger screen and improved camera next month.",
    "sentiment": "positive"
}

> Finished chain.


'{\n    "text": "Apple is releasing a new iPhone with a larger screen and improved camera next month.",\n    "sentiment": "positive"\n}'

## SequentialChain

![Simple Seuqntial Chain](../images/sequential_chain.png)

In [85]:
from langchain.chains import SequentialChain

In [86]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [87]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [88]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [89]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [90]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [91]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': '"I find the taste mediocre. The foam does not hold up, which is strange. I buy the same thing from the store and the taste is much better... Old batch or counterfeit!?"',
 'summary': "The reviewer thinks the taste of the product they received is mediocre and suspects it may be an old batch or counterfeit because the foam doesn't hold up as well as the store-bought version.",
 'followup_message': "Réponse de suivi:\n\nNous sommes désolés d'apprendre que le produit que vous avez reçu n'a pas répondu à vos attentes en termes de goût et de qualité. Nous prenons très au sérieux les préoccupations de nos clients en matière d'authenticité et nous travaillons dur pour nous assurer que tous nos produits sont frais et de la plus haute qualité. Nous aimerions en savoir plus sur votre expérience afin de

## Chain with Memory

In [165]:
# Synopsis
synopsis_template = """
You are a playwright. Given the title of play and the era it is set in, 
it is your job to write a short synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:
"""
synopsis_template = ChatPromptTemplate.from_template(synopsis_template)
synopsis_chain =  LLMChain(llm=llm, prompt=synopsis_template, output_key="synopsis")

In [166]:
# Review
review_template = """
You are a play critic from the New York Times. Given the synopsis of play, 
it is your job to write a short review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
"""
review_template = ChatPromptTemplate.from_template(review_template)
review_chain = LLMChain(llm=llm, prompt=review_template, output_key="review")

In [167]:
social_template = """
You are a social media manager for a theater company.  Given the title of play, 
the era it is set in, the date, time and location, the synopsis of the play, and the review of the play, 
it is your job to write a short social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
social_template = ChatPromptTemplate.from_template(social_template)
social_chain = LLMChain(llm=llm, prompt=social_template, output_key="social_post_text")

In [170]:
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain, social_chain],
    memory=SimpleMemory(memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}),
    input_variables=["era", "title"],
    output_variables=["social_post_text"],
    verbose=True
)

In [171]:
overall_chain({"title":"Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': "Join us on December 25th at 8pm PST at Theater in the Park for a gripping tale of love and duty in Victorian England. Tragedy at Sunset on the Beach follows the story of Elizabeth, torn between her family's expectations and her heart's desire. Don't miss this beautifully crafted play that explores the complexities of love and the consequences of our choices. Critics are raving about the outstanding performances and powerful climax that will leave you deeply affected. Get your tickets now! #TragedyatSunsetontheBeach #TheaterinthePark #VictorianEngland #LoveandDuty #MustSeePlay"}

## Router Chain

![Simple Seuqntial Chain](../images/router_chain.png)

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [55]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [56]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [57]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [58]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [59]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [60]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [61]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [62]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all the radiation incident upon it. A black body is a theoretical object that completely absorbs all radiation that falls on it, reflecting nothing. The spectrum of radiation emitted by a black body depends only on its temperature, and not on its composition or any other feature. As the temperature of the black body increases, the radiation becomes brighter and its peak wavelength shifts towards the shorter (blue) end of the spectrum. This phenomenon is known as Wien’s displacement law. Black body radiation has been used to explain various phenomena in physics, including the observed spectrum of stars and the cosmic microwave background radiation.'

In [63]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'As an AI language model, I am programmed to provide accurate and helpful responses. The answer to your question is 4.'

In [64]:
chain.run("When did Barack Obama serve in the office?")



> Entering new MultiPromptChain chain...
History: {'input': 'When did Barack Obama serve in the office?'}
> Finished chain.


'Barack Obama served as the 44th President of the United States from January 20, 2009 to January 20, 2017.'

In [66]:
#chain.run("Why does every cell in our body contain DNA?")